In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("A3_Q1_train.csv")

In [ ]:
print(df.shape)
df.head()

## Part A
### Task 1

In [ ]:
df_cat = df.drop(columns=['age', 'education_years', 'annual_income'])

In [ ]:
labels = ['low', 'mid', 'high']
age_cat = pd.cut(df['age'], bins=3, labels=labels)
ed_cat = pd.cut(df['education_years'], bins=3, labels=labels)

In [ ]:
df_cat['age_cat'], df_cat['education_years_cat'] = age_cat.astype(object), ed_cat.astype(object)

In [ ]:
df_cat = df_cat.join(df['annual_income'])
df_cat.head()

In [ ]:
df_all_cat = df_cat.copy()

In [ ]:
# so that we can see all the columns
pd.set_option('display.max_columns', None) 
print(df_all_cat.shape)
df_all_cat.head()


In [ ]:
###
# please run below in a separate cell!!!
for col in df_all_cat.columns.tolist():  
    print(col + ':')
    print(df_all_cat[col].value_counts())
    print('********')

### Task 2

In [ ]:
df_all_cat.dtypes

In [ ]:
df_all_cat_ohe = pd.get_dummies(df_all_cat.drop(columns=['annual_income']))

In [ ]:
df_all_cat_ohe['annual_income'] = pd.get_dummies(df['annual_income']).drop(columns='low_income')

In [ ]:
df_all_cat_ohe